In [1]:
# ==========================================
# CSET419 – Introduction to Generative AI
# Lab – 4 Complete Implementation
# ==========================================

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Embedding, LSTM, Dense, Input,
    MultiHeadAttention, LayerNormalization,
    Dropout, GlobalAveragePooling1D
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Layer

# ==========================================
# DATASET
# ==========================================

text = """
artificial intelligence is transforming modern society.
it is used in healthcare finance education and transportation.
machine learning allows systems to improve automatically with experience.
data plays a critical role in training intelligent systems.
large datasets help models learn complex patterns.
deep learning uses multi layer neural networks.
neural networks are inspired by biological neurons.
each neuron processes input and produces an output.
training a neural network requires optimization techniques.
gradient descent minimizes the loss function.

natural language processing helps computers understand human language.
text generation is a key task in nlp.
language models predict the next word or character.
recurrent neural networks handle sequential data.
lstm and gru models address long term dependency problems.
however rnn based models are slow for long sequences.

transformer models changed the field of nlp.
they rely on self attention mechanisms.
attention allows the model to focus on relevant context.
transformers process data in parallel.
this makes training faster and more efficient.
modern language models are based on transformers.

education is being improved using artificial intelligence.
intelligent tutoring systems personalize learning.
automated grading saves time for teachers.
online education platforms use recommendation systems.
technology enhances the quality of learning experiences.

ethical considerations are important in artificial intelligence.
fairness transparency and accountability must be ensured.
ai systems should be designed responsibly.
data privacy and security are major concerns.
researchers continue to improve ai safety.

text generation models can create stories poems and articles.
they are used in chatbots virtual assistants and content creation.
generated text should be meaningful and coherent.
evaluation of text generation is challenging.
human judgement is often required.

continuous learning is essential in the field of ai.
research and innovation drive technological progress.
students should build strong foundations in mathematics.
programming skills are important for ai engineers.
practical experimentation enhances understanding.
"""

# ==========================================
# TOKENIZATION
# ==========================================

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []

for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram = token_list[:i + 1]
        input_sequences.append(n_gram)

max_seq_len = max([len(seq) for seq in input_sequences])

input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')

X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

# ==========================================
# COMPONENT I – LSTM MODEL
# ==========================================

lstm_model = Sequential([
    Embedding(total_words, 100, input_length=max_seq_len - 1),
    LSTM(150),
    Dense(total_words, activation='softmax')
])

lstm_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

print("\nTraining LSTM Model...\n")
lstm_model.fit(X, y, epochs=150, verbose=1)

# ==========================================
# TEXT GENERATION FUNCTION
# ==========================================

def generate_text(model, seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                                   maxlen=max_seq_len - 1,
                                   padding='pre')

        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        seed_text += " " + output_word

    return seed_text


print("\n===== LSTM Generated Text =====")
print(generate_text(lstm_model, "artificial intelligence", 10))
print(generate_text(lstm_model, "transformer models", 10))
print(generate_text(lstm_model, "education is", 10))


# ==========================================
# COMPONENT II – TRANSFORMER MODEL
# ==========================================

class PositionalEncoding(Layer):
    def __init__(self, max_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(max_len, d_model)

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            np.arange(position)[:, np.newaxis],
            np.arange(d_model)[np.newaxis, :],
            d_model
        )

        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]

        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]


def transformer_block(embed_dim, num_heads, ff_dim):
    inputs = Input(shape=(None, embed_dim))

    attention = MultiHeadAttention(
        num_heads=num_heads,
        key_dim=embed_dim
    )(inputs, inputs)

    attention = Dropout(0.1)(attention)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attention)

    ffn = Dense(ff_dim, activation='relu')(out1)
    ffn = Dense(embed_dim)(ffn)
    ffn = Dropout(0.1)(ffn)

    outputs = LayerNormalization(epsilon=1e-6)(out1 + ffn)

    return Model(inputs, outputs)


embed_dim = 64
num_heads = 2
ff_dim = 128

inputs = Input(shape=(max_seq_len - 1,))
x = Embedding(total_words, embed_dim)(inputs)
x = PositionalEncoding(max_seq_len - 1, embed_dim)(x)

transformer = transformer_block(embed_dim, num_heads, ff_dim)
x = transformer(x)

x = GlobalAveragePooling1D()(x)
outputs = Dense(total_words, activation='softmax')(x)

transformer_model = Model(inputs, outputs)

transformer_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nTraining Transformer Model...\n")
transformer_model.fit(X, y, epochs=150, verbose=1)


print("\n===== Transformer Generated Text =====")
print(generate_text(transformer_model, "artificial intelligence", 10))
print(generate_text(transformer_model, "transformer models", 10))
print(generate_text(transformer_model, "education is", 10))


Training LSTM Model...

Epoch 1/150


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.0191 - loss: 5.2740
Epoch 2/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0837 - loss: 5.2520
Epoch 3/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0249 - loss: 5.1912
Epoch 4/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0293 - loss: 4.9977
Epoch 5/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0357 - loss: 4.9094
Epoch 6/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0461 - loss: 4.8779
Epoch 7/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0656 - loss: 4.7559
Epoch 8/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0610 - loss: 4.7878
Epoch 9/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0441 - loss: 4.6922
Epoch 10/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0628 - loss: 4.6399
Epoch 11/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0266 - loss: 4.5654
Epoch 12/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0528 - loss: 4.5032
E